In [ ]:
import cv2
import time
import serial
from ultralytics import YOLO

In [ ]:
# Load the trained YOLOv8 model
model = YOLO("model/best.pt")  # Replace with your actual path


In [ ]:
# Set up serial connection to ESP32
try:
    esp32 = serial.Serial(port='COM6', baudrate=115200, timeout=1)  # Change COM port if needed
    time.sleep(2)
    print("✅ Connected to ESP32")

    esp32.write("START_CONVEYOR\n".encode())
    print("🛠️ Conveyor started")

except serial.SerialException:
    esp32 = None
    print("❌ ESP32 not connected. Detection will still run.")

# Start webcam
cap = cv2.VideoCapture(0)
time.sleep(2)

last_sent = ""

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame)[0]

    # Check for detections
    classes_detected = [model.names[int(cls)] for cls in results.boxes.cls]

    # Determine what to send
    if "bad" in classes_detected:
        signal = "BAD\n"
    elif "good" in classes_detected:
        signal = "GOOD\n"
    else:
        signal = ""

    # Send to ESP32 only if changed
    if signal and signal != last_sent and esp32:
        esp32.write(signal.encode())
        print(f"📤 Sent to ESP32: {signal.strip()}")
        last_sent = signal

    # Show result (optional)
    annotated_frame = results.plot()
    cv2.imshow("Lemon Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
if esp32:
    esp32.close()